In [1]:
import sys
sys.path.append("../")

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scorecardpipeline import *


logger = init_setting(seed=8888, logger=True)

In [2]:
# 加载数据集，标签转换为 0 和 1
target = "creditability"
data = germancredit()
data[target] = data[target].map({"good": 0, "bad": 1})

# 目前仅支持数值型变量
# data = data.select_dtypes("number")
data.head(1)

,status_of_existing_checking_account,duration_in_month,credit_history,purpose,credit_amount,savings_account_and_bonds,present_employment_since,installment_rate_in_percentage_of_disposable_income,personal_status_and_sex,other_debtors_or_guarantors,...,property,age_in_years,other_installment_plans,housing,number_of_existing_credits_at_this_bank,job,number_of_people_being_liable_to_provide_maintenance_for,telephone,foreign_worker,creditability
0,... < 0 DM,6,critical account/ other credits existing (not at this bank),radio/television,1169,unknown/ no savings account,... >= 7 years,4,male : divorced/separated,none,...,real estate,67,none,own,2,skilled employee / official,1,"yes, registered under the customers name",yes,0


In [13]:
rule1 = Rule("duration_in_month < 4")
rule2 = Rule("credit_amount < 500")
rule3 = Rule("purpose.isin(['education', 'business'])")

rules = [rule1, rule2, rule3]

In [14]:
from functools import reduce

for rule in rules:
    display(rule.report(data, target=target))

display(reduce(lambda r1, r2: r1 & r2, rules).report(data, target=target))
display(reduce(lambda r1, r2: r1 | r2, rules).report(data, target=target))

,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,duration_in_month < 4,命中,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7000,0.0000,0.0000,0.0000
1,验证规则,duration_in_month < 4,未命中,1000.0000,1.0000,700.0000,1.0000,300.0000,1.0000,0.3000,1.0000,0.0000,0.3000,0.3000,1.0000,0.4615


,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,credit_amount < 500,命中,18,0.0180,15,0.0214,3,0.0100,0.1667,0.5556,-0.0081,0.6880,0.1667,0.0100,0.0189
1,验证规则,credit_amount < 500,未命中,982,0.9820,685,0.9786,297,0.9900,0.3024,1.0081,0.4444,0.3120,0.3024,0.9900,0.4633


,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,"purpose.isin(['education', 'business'])",命中,147,0.1470,91,0.1300,56,0.1867,0.3810,1.2698,0.0465,0.6650,0.3810,0.1867,0.2506
1,验证规则,"purpose.isin(['education', 'business'])",未命中,853,0.8530,609,0.8700,244,0.8133,0.2860,0.9535,-0.2698,0.3350,0.2860,0.8133,0.4232


,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,"((duration_in_month < 4) & (credit_amount < 500)) & (purpose.isin(['education', 'business']))",命中,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7000,0.0000,0.0000,0.0000
1,验证规则,"((duration_in_month < 4) & (credit_amount < 500)) & (purpose.isin(['education', 'business']))",未命中,1000.0000,1.0000,700.0000,1.0000,300.0000,1.0000,0.3000,1.0000,0.0000,0.3000,0.3000,1.0000,0.4615


,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,"((duration_in_month < 4) | (credit_amount < 500)) | (purpose.isin(['education', 'business']))",命中,162,0.1620,105,0.1500,57,0.1900,0.3519,1.1728,0.0334,0.6520,0.3519,0.1900,0.2468
1,验证规则,"((duration_in_month < 4) | (credit_amount < 500)) | (purpose.isin(['education', 'business']))",未命中,838,0.8380,595,0.8500,243,0.8100,0.2900,0.9666,-0.1728,0.3480,0.2900,0.8100,0.4271


In [5]:
rule1.report(data, target=target)

,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,duration_in_month < 4,命中,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7000,0.0000,0.0000,0.0000
1,验证规则,duration_in_month < 4,未命中,1000.0000,1.0000,700.0000,1.0000,300.0000,1.0000,0.3000,1.0000,0.0000,0.3000,0.3000,1.0000,0.4615


In [6]:
rule2.report(data, target=target)

,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,credit_amount < 500,命中,18,0.0180,15,0.0214,3,0.0100,0.1667,0.5556,-0.0081,0.6880,0.1667,0.0100,0.0189
1,验证规则,credit_amount < 500,未命中,982,0.9820,685,0.9786,297,0.9900,0.3024,1.0081,0.4444,0.3120,0.3024,0.9900,0.4633


In [7]:
rule2.report(data, target=target, prior_rules=rule1)

,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,先验规则,duration_in_month < 4,命中,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7000,0.0000,0.0000,0.0000
1,先验规则,duration_in_month < 4,未命中,1000.0000,1.0000,700.0000,1.0000,300.0000,1.0000,0.3000,1.0000,0.0000,0.3000,0.3000,1.0000,0.4615
0,验证规则,credit_amount < 500,命中,18.0000,0.0180,15.0000,0.0214,3.0000,0.0100,0.1667,0.5556,-0.0081,0.6880,0.1667,0.0100,0.0189
1,验证规则,credit_amount < 500,未命中,982.0000,0.9820,685.0000,0.9786,297.0000,0.9900,0.3024,1.0081,0.4444,0.3120,0.3024,0.9900,0.4633


In [8]:
(rule1 | rule2).report(data, target=target)

,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,(duration_in_month < 4) | (credit_amount < 500),命中,18,0.0180,15,0.0214,3,0.0100,0.1667,0.5556,-0.0081,0.6880,0.1667,0.0100,0.0189
1,验证规则,(duration_in_month < 4) | (credit_amount < 500),未命中,982,0.9820,685,0.9786,297,0.9900,0.3024,1.0081,0.4444,0.3120,0.3024,0.9900,0.4633


In [9]:
(rule1 & rule2).report(data, target=target)

,规则分类,指标名称,分箱,样本总数,样本占比,好样本数,好样本占比,坏样本数,坏样本占比,坏样本率,LIFT值,坏账改善,准确率,精确率,召回率,F1分数
0,验证规则,(duration_in_month < 4) & (credit_amount < 500),命中,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.7000,0.0000,0.0000,0.0000
1,验证规则,(duration_in_month < 4) & (credit_amount < 500),未命中,1000.0000,1.0000,700.0000,1.0000,300.0000,1.0000,0.3000,1.0000,0.0000,0.3000,0.3000,1.0000,0.4615


In [10]:
# writer = ExcelWriter()

# for rule in rules:
#     rule.report()